In [ ]:
import yfinance as yf
from datetime import datetime, timedelta
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import SimpleRNN, LSTM, Dense

# from sklearn.tree import DecisionTreeRegressor
# from xgboost import XGBRegressor

# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# from statsmodels.tsa.arima_model import ARIMA

In [ ]:
tickerSymbol = 'SI=F'
series = yf.Ticker(tickerSymbol)

In [ ]:
prices_full = series.history(start='2024-01-01', end='2024-07-31')
print(prices_full.iloc[-1])

In [ ]:
prices = series.history(start='2024-01-01', end='2024-07-31').Open
returns = prices.pct_change().dropna()

In [ ]:
print(prices)

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(prices)
plt.show()

In [ ]:
baseline_profit = prices.iloc[-1] - prices.iloc[0]
baseline_profit = np.round(baseline_profit,2)
print("Baseline Profit:  $" + str(baseline_profit) + " per contract")

In [ ]:
print(str(100) + "/" + str(prices.iloc[0]) + " = " + "x/$" + str(baseline_profit))

In [ ]:
print("Profit to beat:  $17.38")

In [ ]:
window_size = 5

data_train = prices[:'2024-06-30'].to_numpy()
data_test = prices['2024-06-24':].to_numpy()


X_train = []
y_train = []
for i in range(len(data_train) - window_size):
    X_train.append(data_train[i: i + window_size])
    y_train.append(data_train[i+window_size])

X_test = []
y_test = []
for i in range(len(data_test) - window_size):
    X_test.append(data_test[i: i + window_size])
    y_test.append(data_test[i+window_size])

X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
baseline_profit_test = data_test[-1] - data_test[0]
baseline_profit_test = np.round(baseline_profit_test,2)
print("Baseline Profit (Test):  $" + str(baseline_profit_test) + " per contract")

In [ ]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [ ]:
print('X_train shape = ', X_train.shape)
print('y_train shape = ', y_train.shape,'\n')

print('X_test shape = ', X_test.shape)
print('y_test shape = ', y_test.shape)

In [ ]:
model = Sequential()
model.add(SimpleRNN(128, activation='relu', return_sequences=True, input_shape=(window_size, 1)))
model.add(SimpleRNN(256, activation='relu', return_sequences=True, input_shape=(window_size, 1)))
model.add(SimpleRNN(128, activation='relu', return_sequences=True, input_shape=(window_size, 1)))
model.add(SimpleRNN(64, activation='relu'))
model.add(Dense(1))

In [ ]:
model.compile(loss='mse', optimizer='adam')
model.fit(np.array(X_train), np.array(y_train), epochs=5000, verbose=1)

In [ ]:
model.summary()

In [ ]:
results = model.predict(np.array(X_test))

In [ ]:
pred_series = pd.Series(results.flatten(), index=prices['2024-06-24':].index[window_size:window_size + len(results)])

In [ ]:
prices_2024 = prices['2024-06-01':]
plt.figure(figsize=(10, 5))
plt.plot(prices_2024, color='blue', label='Actual Prices')
plt.plot(pred_series, color='red', label='Predicted Prices')
plt.legend()
plt.xlabel('Year-Month')
plt.ylabel('Price')
plt.title('Gold Price vs Time')
plt.grid(True)
plt.show()

In [ ]:
trading_period = data_test
predictions = results

prices_test = prices['2024-06-24':]

prices_trading = prices['2024-06-28':]
print(prices_trading)

In [ ]:
print(pred_series)

In [ ]:
def BuySellModel(starting_cash, prices=prices_trading, predictions=pred_series):
    baseline_revenue = prices[-1]*int(starting_cash / prices[0]) - prices[0]*int(starting_cash / prices[0])
    cash = starting_cash
    contracts = None
    holding = False
    buy_price = None
    sell_price = None

    for i in range(len(predictions)):
        if not holding:
            # if prediction is >> todays price -> Buy as many contracts
            if predictions[i] > prices[i]:
                contracts = int(cash / prices[i])
                cash -= prices[i]*contracts
                holding = True
        else:
            # if prediction is << todays price -> Sell as many contracts
            # force sell at final day
            if predictions[i] < prices[i] or i == len(predictions)-1:
                cash += prices[i]*contracts
                contracts = 0
                holding = False
    
    revenue = cash - starting_cash
    revenue_percent = (revenue * 100)/starting_cash
    return cash, revenue, revenue_percent, baseline_revenue

In [ ]:
cash, revenue, revenue_percent, baseline_revenue = BuySellModel(starting_cash=10000)
print("Current Cash:", cash)
print("Revenue:", revenue)
print("Baseline Revenue:", baseline_revenue)
if revenue_percent > 0:
    print(str(revenue_percent)+"% Profit\n")
else:
    print(str(revenue_percent)+"% Loss\n")


In [ ]:
def BuySellModelPredict(starting_cash):
    prices = trading_period
    revenue = np.zeros(len(predictions))
    cash = starting_cash
    cash_used = 0
    shares = 0
    holding = False

    for i in range (1, len(predictions)):
        if predictions[i-1] > 0:
            shares = starting_cash / prices[i-1]
            cash = shares * prices[i]
            cash_used += starting_cash
            revenue[i] = cash - starting_cash
            holding = True

        print(f"Day {i}: \n Daily Open = {prices[i-1]},  \n Daily Close = {prices[i]},  \n Cash = {cash}, \n Shares = {shares}, \n Prediction = {predictions[i-1]}, \n Daily Revenue = {revenue[i]}, \n")

    if holding:
        cash = shares * prices[-1]
        revenue[-1] = cash - starting_cash
        print(f"Day {i+1}: Forced Sell: Close = {prices[-1]}, Cash = {cash}, Revenue = {revenue[-1]}")

    #   plt.figure(figsize=(10, 4))
    #   plt.plot(revenue)
    #   plt.ylabel('Revenue', fontsize=20)
    #   plt.xlabel('Hour', fontsize=20)
    #   plt.title('Revenue Over Time', fontsize=20)
    #   plt.show()

    plt.figure(figsize=(10, 4))
    days = range(len(revenue))
    plt.scatter(days, revenue, color='blue', label='Revenue')
    plt.plot(days, revenue, color='blue')
    plt.ylabel('Revenue', fontsize=20)
    plt.xlabel('Day', fontsize=20)
    plt.title('Revenue Over Time', fontsize=20)
    plt.legend()

    # Set x-axis ticks at intervals of 24
    xticks = range(0, len(revenue), 24)
    plt.xticks(xticks)

    # Annotate each point with the revenue value
    for day, rev in zip(days, revenue):
        plt.text(day, rev, f'{round(rev, 2)}', fontsize=7, ha='left', va='bottom')

    plt.show()
    total_revenue = 0
    for i in range(len(revenue)):
        total_revenue += revenue[i]
    print('Cash Used: ', cash_used)
    return total_revenue
BuySellModelPredict(starting_cash=100)